In [6]:
from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector import Error
import pandas as pd

In [7]:
dbconfig = {
    'user': 'root',
    'password': '******',
    'database' :'meta_pytomysql_test'
    }

try:
    pool = MySQLConnectionPool(
        pool_name= 'meta_pytomysql_test_pool',
        pool_size= 4,
        **dbconfig
        )
    print('The connection pool is created with a name: ', pool.pool_name)
    print('The pool size is: ', pool.pool_size)
    
except Error as err:
    print('Error code: ', err.errno)
    print('Error message: ', err.msg)

The connection pool is created with a name:  meta_pytomysql_test_pool
The pool size is:  4


In [8]:
# Insert a new user and establish a new connection:

connection1 = pool.get_connection()
print(f'Connection estabilished. {connection1.connection_id} is the user ID of the connection.')

Connection estabilished. 82 is the user ID of the connection.


In [9]:
# Create cursor using the new connection:

try:
    cursor1 = connection1.cursor()
    print('Cursor created.')
except:
    print('Not possible to create tehe cursor')

Cursor created.


In [12]:
# 1. Create and call a stored procedure named PeakHours that identifies the peak, or busiest hour, for the restaurant based on the number of bookings.

cursor1.execute("DROP PROCEDURE IF EXISTS peakhours")

query_peakhours = """
CREATE PROCEDURE peakhours()
BEGIN
SELECT COUNT(*)  AS 'Quantity', HOUR(bookingSlot) AS 'Hour' 
FROM bookings 
GROUP BY Hour 
ORDER BY Quantity DESC;
END
"""
cursor1.execute(query_peakhours)
print('Procedure created.')

Procedure created.


In [13]:
# 1.1 Call and create a dataframe for the created procedure:

cursor1.callproc('peakhours')
results1 = next(cursor1.stored_results())
dataset1 = results1.fetchall()
columns1 = results1.column_names

df1 = pd.DataFrame(data=dataset1, columns=columns1)
df1.style.hide(axis='index')

Quantity,Hour
2,19
1,15
1,17
1,18
1,20


In [15]:
# 2. Create and call a stored procedure named GuestStatus that outputs status of each guest’s order based on which employee is assigned to the order.
"""
If the Role in the Employee table is Manager or Assistant Manager then the guest’s order status is Ready to pay

If the Role in the Employee table is Head Chef then the status is Ready to serve

If the Role in the Employee table is Assistant Chef then the status is Preparing Order

If the Role in the Employee table is Head Waiter then the status is Order served
"""

cursor1.execute("DROP PROCEDURE IF EXISTS GuestStatus")

query_GuestStatus  = """
CREATE PROCEDURE GuestStatus()
BEGIN

SELECT b.tableNo, CONCAT(b.guestFirstName, ' ',b.guestLastName) 'Name',
CASE
    WHEN e.role = 'Manager' OR e.role = 'Assistant Manager' THEN 'Ready to pay'
    WHEN e.role = 'Head Chef' THEN 'Ready to serve'
    WHEN e.role = 'Assistant Chef' THEN 'Preparing Order'
    WHEN e.role = 'Head Waiter' THEN 'Order served'
END AS Status    
FROM bookings b LEFT JOIN employees e 
ON b.employeeID = e.employeeID;
END
"""
cursor1.execute(query_GuestStatus)
print('Procedure created.')

Procedure created.


In [16]:
# 2.1 Call and create a dataframe for the created procedure:

cursor1.callproc('GuestStatus')
results2 = next(cursor1.stored_results())
dataset2 = results2.fetchall()
columns2 = results2.column_names

df2 = pd.DataFrame(data=dataset2, columns=columns2)
df2.style.hide(axis='index')

tableNo,Name,Status
12,Anna Iversen,Ready to pay
12,Joakim Iversen,Ready to pay
19,Vanessa McCarthy,Ready to serve
15,Marcos Romero,Preparing Order
5,Hiroki Yamane,Ready to pay
8,Diana Pinto,Order served
